<H2>IMPORTING LIBRARIES<H2>

In [89]:
import os
import cv2
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
import numpy as np
from keras.layers.core import Lambda, Flatten, Dense
from keras.models import load_model
import keras.backend as K
import dlib
import matplotlib.pyplot as plt
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import imutils
from numpy import load
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import pickle
import tensorflow as tf    
tf.random.set_seed(1234)

<h2>FACE DETECTION USING DLIB</h2>

In [91]:
shape_predictor_path="shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(shape_predictor_path)
fa = FaceAligner(predictor, desiredFaceWidth=160)

In [85]:
detector = dlib.get_frontal_face_detector()
def detect_face_align_images(image_path,type_of_app):
# load the input image, resize it, and convert it to grayscale
    if type_of_app!='stream':
        path=image_path
        img = cv2.imread(path)
    else:
        img = image_path
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1) # result
    bbox=[]
    xywh={}
    #to draw faces on image
    for result in faces:
        x = result.left()
        y = result.top()
        x1 = result.right()
        y1 = result.bottom()
        xywh['x']=x
        xywh['y']=y
        xywh['x1']=x1
        xywh['y1']=y1
        bbox.append(xywh)
    if len(bbox) > 0:
        for bb in bbox:
            rect = dlib.rectangle(bb['x'], bb['y'], bb['x1'], bb['y1'])
            faceAligned = fa.align(img, gray, rect)
            return faceAligned,bbox,img
    else:
        return [],[],[]

<h2>CREATING THUMBNAILS 160*160 FOR EMBEDINGS</h2>
<p>Whole application ran in windows system so paths might create an issue.Please check out the paths while you run on linux machine<p>

In [33]:
def create_thumbnail(faces):
    faces_path=faces 
    #path to original images
    images=[]
    # to store paths of images from folder
    thumbnail_path = 'thumbnail_image'
    for subdir in os.listdir(faces_path):
        path=os.path.join(faces_path,subdir)
        for im in os.listdir(path):
            if '.ipynb_checkpoints' not in im:
                images.append(os.path.join(path,im))
    for image_path in images:
        try:
            label = image_path.split('\\')[1]
            thumbnail,bbox,image=detect_face_align_images(image_path)
            save_path = os.path.join(thumbnail_path,label)
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            new_image_path= save_path+'/'+label+"_"+image_path.split('\\')[2]+".png"
            cv2.imwrite(new_image_path,thumbnail)
        except Exception as error:
            print(error)

    return thumbnail_path
create_thumbnail('faces')

'thumbnail_image'

<h2>EMBEDDING CREATION</h2>
<p>Whole application ran in windows system so paths might create an issue.Please check out the paths while you run on linux machine</p>

In [94]:

def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    return yhat[0]

def load_dataset(dir):
    X, y = list(), list()
    faces=[]
    labels=[]
    for subdir in os.listdir(dir):
        path=os.path.join(dir,subdir)
        for im in os.listdir(path):
            if '.ipynb_checkpoints' not in im:
                file_path=os.path.join(path,im)
                faces.append(cv2.imread(file_path))
                labels.append(file_path.split('\\')[1])
    X.extend(faces)
    y.extend(labels)
    return np.asarray(X), np.asarray(y)

<h2>Using only 3 faces for Training such that Train and Test split is done by taking odd and even faces</h2>

In [96]:
def create_embeddings(thumbnail_faces):
    model = load_model('facenet_keras.h5')
    trainX, trainy = load_dataset(thumbnail_faces)
    aligned_thumb_emb_path=''
    np.savez_compressed('Embedings/embed_align.npz', trainX, trainy)    
    data = np.load('Embedings/embed_align.npz')
    trainX, trainy = data['arr_0'], data['arr_1']
    X_train=[]
    X_test=[]
    y_train=[]
    y_test=[]
    for key,train_test_split in enumerate(trainX):
        if key % 2==0:
            X_train.append(trainX[key])
            y_train.append(trainy[key])
        else:
            X_test.append(trainX[key])
            y_test.append(trainy[key])
    emdTrainX = list()
    emdTestX = list()
    try:
        for face in X_train:
            emd = get_embedding(model, face)
            emdTrainX.append(emd)

        emdTrainX = np.asarray(emdTrainX)
        for face in X_test:
            emd = get_embedding(model, face)
            emdTestX.append(emd)
        emdTestX = np.asarray(emdTestX)     
    except Exception as err:
        print(err)
    np.savez_compressed('Embedings/embed_train.npz', emdTrainX, y_train, emdTestX, y_test)
    return emdTrainX,emdTestX
create_embeddings('thumbnail_image')

(array([[ 3.08030844e-01, -2.97826678e-01,  6.42760247e-02,
          1.95717001e+00,  2.99809307e-01,  1.14060259e+00,
          9.85327810e-02,  1.02672303e+00,  1.52338111e+00,
         -2.08603472e-01, -2.45661092e+00, -1.17160702e+00,
          3.59397709e-01, -2.45626211e-01, -1.59880149e+00,
          9.45915878e-02,  1.74916017e+00, -1.26266682e+00,
          6.29114032e-01, -6.84681773e-01,  8.04378390e-02,
          1.10844839e+00, -1.04231572e+00,  1.93230838e-01,
         -7.20081180e-02, -1.64775062e+00,  3.22061956e-01,
          2.86462128e-01,  4.10504103e-01, -5.01221418e-01,
         -4.82397407e-01, -2.64044017e-01,  2.40948439e-01,
         -1.16990221e+00,  4.93515611e-01, -1.12311018e+00,
         -1.33955431e+00,  1.04885447e+00,  4.79024827e-01,
          5.25918864e-02, -1.77011359e+00,  3.27188164e-01,
         -1.74863636e-01,  1.65204585e-01, -1.58854779e-02,
          4.53019053e-01, -1.26507831e+00,  1.61072445e+00,
         -7.32037425e-03,  3.29074323e-0

<h2>Load Embeddings  & Train Classifier</h2>

In [46]:
data = np.load('Embedings/embed_train.npz')
X_train, trainy, X_test, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

# Encode the labels
le = LabelEncoder()
train_labels = le.fit_transform(trainy)
test_labels= le.fit_transform(testy)
num_classes = len(np.unique(train_labels))
train_labels = train_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)
#converting clases to categorical variables
Y_train = np_utils.to_categorical(train_labels, num_classes)
Y_test = np_utils.to_categorical(test_labels, num_classes)
input_shape = (128,)

# building a linear stack of layers with the sequential model
model = Sequential()
# hidden layer
model.add(Dense(512, activation='relu', input_shape=input_shape))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# looking at the model summary
model.summary()
# compiling the sequential model
# training the model for 30 epochs
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=32, epochs=30, validation_data=(X_test, Y_test))
#saving Model
model.save('Face_recogniser.h5')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               66048     
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 1539      
Total params: 330,243
Trainable params: 330,243
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
1/1 [==============================] - 0s 237ms/step - loss: 1.2928 - accuracy: 0.2000 - val_loss: 1.22

In [92]:
#Loading Facenet and Classifier
face_recognise_model=load_model("Face_recogniser.h5")
facenet_model = load_model('facenet_keras.h5')

<h2>Calculating Cosine Similarity for reducing False Positives <h2>

In [50]:
def findCosineDistance(vector1, vector2):
    """
    Calculate cosine distance between two vector
    """
    vec1 = vector1.flatten()
    vec2 = vector2.flatten()
    a = np.dot(vec1.T, vec2)
    b = np.dot(vec1.T, vec1)
    c = np.dot(vec2.T, vec2)
    return 1 - (a/(np.sqrt(b)*np.sqrt(c)))

def CosineSimilarity(test_vec, source_vecs):
    """
    Verify the similarity of one vector to group vectors of one class
    """
    cos_dist = 0
    for source_vec in source_vecs:
        cos_dist += findCosineDistance(test_vec, source_vec)
    return cos_dist/len(source_vecs)

<h2>Streaming Recogniser</h2>

In [105]:
cap = cv2.VideoCapture(0)
type_of_app='stream'
embedded_dim=128
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Recog_20201224_130715648_test.avi',fourcc, 5,(640,480))
while True:
    ret, frames = cap.read()
    frame =cv2.resize(frames,(640,480),fx=0.0,fy=0.0, interpolation = cv2.INTER_CUBIC)
    images,bbox,frames = detect_face_align_images(frame,type_of_app)
    if len(bbox) != 0:
        for num,b in enumerate(bbox):
            gray = cv2.cvtColor(frames, cv2.COLOR_BGR2GRAY)
            confidence_scores=[]
            bb = dlib.rectangle(bbox[num]['x'],bbox[num]['y'],bbox[num]['x1'],bbox[num]['y1'])
            thumbnail = images.astype('float32')
            aligned_thumbnail=thumbnail.copy()
            face_img=thumbnail
            mean, std = thumbnail.mean(), thumbnail.std()
            face_img = (thumbnail-mean)/std
            vector = facenet_model.predict(np.expand_dims(face_img, axis=0))[0]
            # Predict class
            preds = model.predict(vector.reshape(1,-1))
            preds = preds.flatten()
            # Get the highest accuracy embedded vector
            j = np.argmax(preds)
            proba = preds[j]
            match_class_idx=np.where(trainy==le.classes_[j])[0]
            match_class_idxs = np.where(match_class_idx)[0]
            selected_idx = np.random.choice(match_class_idx, 5)
            compare_embeddings = X_train[selected_idx]
            # Calculate cosine similarity
            cos_similarity = CosineSimilarity(vector, compare_embeddings)
            text = "Unknown"
            if cos_similarity < 0.50 and proba > 0.90:
                name = le.classes_[j]
                text = "{}".format(name)
                text = str("{}: {:.2f}%".format(text, proba*100))
                print("Recognized: {} <{:.2f}>".format(name, proba*100))
                y = bbox[num]['y'] - 10 if bbox[num]['y'] - 10 > 10 else bbox[num]['y'] + 10
                cv2.putText(frame,text, (bbox[num]['x'], y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frame, (bbox[num]['x'], bbox[num]['y']), (bbox[num]['x1'], bbox[num]['y1']), (0,0,255), 3)
#     b = cv2.resize(frame,(256,256),fx=0.0,fy=0.0, interpolation = cv2.INTER_CUBIC)
    out.write(frame)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Recognized: GB <99.78>
Recognized: GB <99.82>
Recognized: GB <99.82>
Recognized: GB <99.87>
Recognized: GB <99.64>
Recognized: GB <99.80>
Recognized: GB <99.78>
Recognized: GB <99.90>
Recognized: GB <99.86>
Recognized: GB <99.83>
Recognized: GB <99.84>
Recognized: GB <99.36>
Recognized: GB <99.57>
Recognized: GB <99.82>
Recognized: GB <99.59>
Recognized: GB <99.89>
Recognized: GB <99.85>
Recognized: GB <99.86>
Recognized: GB <99.61>
Recognized: GB <99.42>
Recognized: GB <99.70>
Recognized: GB <99.85>
Recognized: GB <99.93>
Recognized: GB <99.70>
Recognized: GB <99.32>
Recognized: GB <99.81>
Recognized: GB <97.06>
Recognized: GB <90.80>
Recognized: GB <97.37>
Recognized: GB <99.16>
Recognized: GB <97.49>
Recognized: GB <97.38>
Recognized: GB <96.78>
Recognized: GB <96.66>
Recognized: GB <93.86>
Recognized: GB <91.14>
Recognized: GB <97.72>
Recognized: GB <94.22>
Recognized: GB <91.79>
Recognized: GB <95.99>
Recognized: GB <90.36>
Recognized: GB <96.06>
Recognized: GB <96.21>
Recognized: